# Deploying and invoking Drift Detector Dataflow Flex template

In [1]:
%cd drift_detector

/home/jarekk/repos/drift-monitor/drift_detector


In [2]:
import json
from datetime import datetime
import time

## Build the Flex template docker image

In [3]:
template_name = 'drift-detector'

project_id = !(gcloud config get-value core/project)
project_id = project_id[0]
template_image = 'gcr.io/{}/{}:latest'.format(project_id, template_name)
location = 'us-central1'

In [4]:
!gcloud builds submit --tag {template_image} .

) (3.7.4.1)
  Created wheel for absl-py: filename=absl_py-0.8.1-py3-none-any.whl size=121165 sha256=8ef6dc8ccbf1ae5011a9a8f578f3e3009a04ff949180c0ec639c9287fba64ef0
  Stored in directory: /root/.cache/pip/wheels/0c/fa/af/c3217c654740d2cffc40ab4eca9da558be4bb94dd593ef0d74
  Created wheel for httplib2: filename=httplib2-0.12.0-py3-none-any.whl size=93464 sha256=f9eda2f71d3de72c9ce879ed0f1d40fd38e9c6aeefe56ced652e690b34057045
  Stored in directory: /root/.cache/pip/wheels/2f/d8/97/b0a618545428f3602f565ccc1309f7cdeb8b4916ad8ec43685
  Created wheel for backcall: filename=backcall-0.1.0-py3-none-any.whl size=10413 sha256=4cfb16e445f916dbc199ecbe0d6ce533ab9c9391f6b6d118fb438a0e3e83b818
  Stored in directory: /root/.cache/pip/wheels/cb/6f/5c/9b66a87adeb46fed2babb14407bae1271f5e41b6b34b642411
Successfully built absl-py httplib2 backcall
ERROR: tensorflow-serving-api 2.1.0 has requirement tensorflow~=2.1.0, but you'll have tensorflow 2.2.0 which is incompatible.
  Attempting uninstall: pyarrow
 

## Deploy the Flex template using `gcloud`

In [5]:
flex_templates_location = 'gs://mlops-dev-workspace/flex-templates'
template_path = '{}/{}.json'.format(flex_templates_location, template_name)
metadata_file = 'metadata.json'

In [6]:
!gcloud beta dataflow flex-template build {template_path} \
  --image {template_image} \
  --sdk-language "PYTHON" \
  --metadata-file {metadata_file} \
  --image {template_image} \
  --sdk-language "PYTHON" \
  --metadata-file {metadata_file}

Successfully saved container spec in flex template file.
Template File GCS Location: gs://mlops-dev-workspace/flex-templates/drift-detector.json
Container Spec:

{
    "image": "gcr.io/mlops-dev-env/drift-detector:latest",
    "metadata": {
        "description": "Data drift detector Python flex template.",
        "name": "Data drift detector Python flex template",
        "parameters": [
            {
                "helpText": "A full name of the BQ request-response log table",
                "label": "Request response log table.",
                "name": "request_response_log_table",
                "regexes": [
                    "[-_.a-zA-Z0-9]+"
                ]
            },
            {
                "helpText": "A type of instances in request_response log_table",
                "label": "Instance type.",
                "name": "instance_type",
                "regexes": [
                    "LIST|OBJECT"
                ]
            },
            {
              

## Trigger a run of the template using REST API

In [7]:
import googleapiclient.discovery

service = googleapiclient.discovery.build('dataflow', 'v1b3')

parameters = {
    'request_response_log_table': 'mlops-dev-env.data_validation.covertype_classifier_logs_tf',
    'instance_type': 'OBJECT',
    'start_time': '2020-05-23T17:00:00',
    'end_time': '2020-05-23T23:00:00',
    'output_path': 'gs://mlops-dev-workspace/drift_monitor/output/tf/test1',
    'schema_file': 'gs://mlops-dev-workspace/drift_monitor/schema/schema.pbtxt',
    'setup_file': './setup.py',

}
job_name = "data-drift-{}".format(time.strftime("%Y%m%d-%H%M%S"))
body = {
    'launch_parameter': 
         {
             'jobName': job_name,
             'parameters' : parameters,
             'containerSpecGcsPath': template_path
         }}

request = service.projects().locations().flexTemplates().launch(
    location=location,
    projectId=project_id,
    body=body)

response = request.execute()
response

{'job': {'id': '2020-05-24_08_39_24-9242796634023112412',
  'projectId': 'mlops-dev-env',
  'name': 'data-drift-20200524-153924',
  'currentStateTime': '1970-01-01T00:00:00Z',
  'createTime': '2020-05-24T15:39:25.462526Z',
  'location': 'us-central1',
  'startTime': '2020-05-24T15:39:25.462526Z'}}

## Trigger a run of the template using REST API - Curl

In [ ]:
service_url = 'https://dataflow.googleapis.com/v1b3/projects/{}/locations/us-central1/flexTemplates:launch'.format(project_id)
headers_content = 'Content-Type: application/json'
access_token = !(gcloud auth print-access-token) 
headers_auth = 'Authorization: Bearer {}'.format(access_token[0])
parameters = {
    'request_response_log_table': 'mlops-dev-env.data_validation.covertype_classifier_logs_tf',
    'instance_type': 'OBJECT',
    'start_time': '2020-05-15T00:15:00',
    'end_time': '2020-05-15T05:51:00',
    'output_path': 'gs://mlops-dev-workspace/drift_monitor/output/tf/test2',
    'schema_file': 'gs://mlops-dev-workspace/drift_monitor/schema/schema.pbtxt',
    'setup_file': './setup.py',

}
job_name = "data-drift-{}".format(time.strftime("%Y%m%d-%H%M%S"))
body = {
    'launch_parameter': 
         {
             'jobName': job_name,
             'parameters' : parameters,
             'containerSpecGcsPath': template_path
         }}

json_body = json.dumps(body)

In [ ]:
!curl -X POST \
  "{service_url}" \
  -H "{headers_content}" \
  -H "{headers_auth}" \
  -d '{json_body}'

## Trigger a run of the template using `gcloud`

In [ ]:
job_name = "data-drift-{}".format(time.strftime("%Y%m%d-%H%M%S"))
g_parameters = ','.join(['{}={}'.format(key,value) for key, value in parameters.items()])

!gcloud beta dataflow flex-template run {job_name} \
--template-file-gcs-location {template_path} \
--parameters {g_parameters}